In [ ]:
%matplotlib inline
import random, pylab, math, random

# Introduction

In this week's lecture, we studied three algorithms for the simulation of hard disks in a box:
* Event-driven molecular dynamics
* Direct sampling
* Markov-chain sampling

In this homework, you will check that the statistical properties of these algorithms are all the same. You will then interpret their output, and finally, explicitly verify the equiprobability principle for hard disks in a box.

# A

In this section, we consider four disks in a square box of edge length 1, in a box without periodic boundary conditions. We set the density equal to $\eta = 0.18$, which corresponds to a disk radius $\sigma \simeq 0.1196$.

We consider a simple observable, the position $x$, as the $x$-coordinate of the center of a disk. We will compute its probability distribution, as the normed histogram of $x$-positions. This histogram is the same for all disks, so we can collect data for one disk or for all of them.

## A1

Compute the histogram of the $x$-positions for the direct-sampling Monte Carlo algorithm by modifying the algorithm implemented in the cell below. For the sake of students without prior knowledge of python, the corresponding code is given below . Simply run it to produce a histogram of the x-positions with 100 bins as output.

In [ ]:
def direct_disks_box(N, sigma):
    overlap = True
    while overlap == True:
        L = [(random.uniform(sigma, 1.0 - sigma), random.uniform(sigma, 1.0 - sigma))]
        for k in range(1, N):
            a = (random.uniform(sigma, 1.0 - sigma), random.uniform(sigma, 1.0 - sigma))
            min_dist_sq = min(((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) for b in L) 
            if min_dist_sq < 4.0 * sigma ** 2: 
                overlap = True
                break
            else:
                overlap = False
                L.append(a)
    return L

N = 4
sigma = 0.1197
n_runs = 1000000
histo_data = []
for run in range(n_runs):
    pos = direct_disks_box(N, sigma)
    for k in range(N): histo_data.append(pos[k][0])

In [ ]:
pylab.hist(histo_data, bins=100, normed=True)
pylab.xlabel('x')
pylab.ylabel('frequency')
pylab.title(r'Direct sampling, $10^6$ valid configs')
pylab.grid()
pylab.savefig('direct_disks_histo.png')
pylab.show()

## A2
### A2.1

Compute the histogram of the $x$-positions with 100 bins for the Markov-chain sampling algorithm, by
modifying the algorithm presented in class, and repeated in the cell below, in a manner similar to what was done in section A1. Set n_steps=2000000.

In [ ]:
L = [[0.25, 0.25], [0.75, 0.25], [0.25, 0.75], [0.75, 0.75]]
sigma = 0.15
sigma_sq = sigma ** 2
delta = 0.1
n_steps = 1000
for steps in range(n_steps):
    a = random.choice(L)
    b = [a[0] + random.uniform(-delta, delta), a[1] + random.uniform(-delta, delta)]
    min_dist = min((b[0] - c[0]) ** 2 + (b[1] - c[1]) ** 2 for c in L if c != a)
    box_cond = min(b[0], b[1]) < sigma or max(b[0], b[1]) > 1.0 - sigma
    if not (box_cond or min_dist < 4.0 * sigma ** 2):
        a[:] = b
print L

### A2.2

State the **three mathematical conditions** that guarantee that the histogram for the Markov-chain
sampling algorithm agrees with the direct-sampling one. Furthermore, explain briefly whether
and why these three conditions are in fact satisfied by the four-disk system at density $\eta = 0.18$.

## A3 Molecular dynamics

We now consider the event driven molecular dynamics algorithm. In sections (A3.1), (A3.2), and (A3.3), always
start from the algorithm provided in the cell below.

In [ ]:
def wall_time(pos_a, vel_a, sigma):
    if vel_a > 0.0:
        del_t = (1.0 - sigma - pos_a) / vel_a
    elif vel_a < 0.0:
        del_t = (pos_a - sigma) / abs(vel_a)
    else:
        del_t = float('inf')
    return del_t

def pair_time(pos_a, vel_a, pos_b, vel_b, sigma):
    del_x = [pos_b[0] - pos_a[0], pos_b[1] - pos_a[1]]
    del_x_sq = del_x[0] ** 2 + del_x[1] ** 2
    del_v = [vel_b[0] - vel_a[0], vel_b[1] - vel_a[1]]
    del_v_sq = del_v[0] ** 2 + del_v[1] ** 2
    scal = del_v[0] * del_x[0] + del_v[1] * del_x[1]
    Upsilon = scal ** 2 - del_v_sq * ( del_x_sq - 4.0 * sigma **2)
    if Upsilon > 0.0 and scal < 0.0:
        del_t = - (scal + math.sqrt(Upsilon)) / del_v_sq
    else:
        del_t = float('inf')
    return del_t

pos = [[0.25, 0.25], [0.75, 0.25], [0.25, 0.75], [0.75, 0.75]]
vel = [[0.21, 0.12], [0.71, 0.18], [-0.23, -0.79], [0.78, 0.1177]]
singles = [(0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1), (3, 0), (3, 1)]
pairs = [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
sigma = 0.15
t = 0.0
n_events = 100
for event in range(n_events):
    wall_times = [wall_time(pos[k][l], vel[k][l], sigma) for k, l  in singles]
    pair_times = [pair_time(pos[k], vel[k], pos[l], vel[l], sigma) for k, l in pairs]
    next_event = min(wall_times + pair_times)
    t += next_event
    for k, l in singles: pos[k][l] += vel[k][l] * next_event 
    if min(wall_times) < min(pair_times):
        collision_disk, direction = singles[wall_times.index(next_event)]
        vel[collision_disk][direction] *= -1.0
    else: 
        a, b = pairs[pair_times.index(next_event)]
        del_x = [pos[b][0] - pos[a][0], pos[b][1] - pos[a][1]]
        abs_x = math.sqrt(del_x[0] ** 2 + del_x[1] ** 2)
        e_perp = [c / abs_x for c in del_x]
        del_v = [vel[b][0] - vel[a][0], vel[b][1] - vel[a][1]]
        scal = del_v[0] * e_perp[0] + del_v[1] * e_perp[1]
        for k in range(2): 
            vel[a][k] += e_perp[k] * scal 
            vel[b][k] -= e_perp[k] * scal 
    print 'event', event
    print 'time', t
    print 'pos', pos
    print 'vel', vel

### A3.1.1

Compute the histogram for the $x$-positions with 100 bins analogously to what you did in sections (A1) and (A2.1),
but for the event-driven molecular dynamics algorithm. Collect the histogram from all particles and collision times (wall and pair collisions) and use a total number of events n_events=200000.

### A3.1.2

This histogram differs from the ones in sections (A1) and (A2.1). Briefly explain its outstanding feature: the large
probability at $x = \sigma$ and $x = 1 - \sigma$.

### A3.2

You may think that the difference between Monte Carlo and Molecular dynamics comes from taking into
account wall collisions. To test this idea, compute the histogram of the x-positions for the event-driven
molecular dynamics algorithm, but only at ALL PAIR COLLISION TIMES (that is, drop the wall collisions from (A3.1.1)) for n_events = 500000. 

### A3.3.1
Finally, compute the histogram of the x-positions with 100 bins for the event-driven
molecular dynamics algorithm, where you take the $x$-positions at regular time intervals $t=0,1,2,3,\ldots$, using the
following code snippet, to be introduced after the line

"next_event = min(wall_times + pair_times)":

In [ ]:
next_event = min(wall_times + pair_times)
t_previous = t
for inter_times in range(int(t + 1), int(t + next_event + 1)):
    del_t = inter_times - t_previous
    for k, l in singles:
        pos[k][l] += vel[k][l] * del_t
    t_previous = inter_times
    for k in range(4):
        histo_data.append(pos[k][0])
t += next_event

Further, replace the line

In [ ]:
for k, l in singles: pos[k][l] += vel[k][l] * next_event

in the original algorithm by the line:

In [ ]:
for k, l in singles: pos[k][l] += vel[k][l] * (t - t_previous)

Set n_events=1000000. Compare the histogram with the ones you obtained in sections (A1) and (A2.1). Are
the probabilities the same?

### A3.3.2

The histogram you obtained in section (A3.3.1) should be (up to statistical fluctuations) identical to what
you obtained in sections (A1) and (A2.1). This is the consequence of the equivalence between Newton's deterministic dynamics and Boltzmann's statistical dynamics. Does this equivalence follow from the detailed balance condition respected by the algorithm?

# B

In this week's lecture, we have emphasized the importance of the equiprobability principle, which governs the statistical physics of hard disks and hard spheres, yet we have observed a manifestly non-uniform probability distribution of the $x$-positions (the equiprobability principle manifestly does not apply to $x$-positions).

## B1

To convince yourself that equiprobability **is** satisfied, in the eight-dimensional space of allowed
configurations of four-disk positions, consider the configuration (A) shown in the figures below. The
probability to sample this configuration **exactly** is of course zero, so we must put little boxes around this
configuration, as illustrated by the red squares.

![Configuration A](./Figures/A_fig.png "Configuration A")

![Configuration B](./Figures/B_fig.png "Configuration B")

![Configuration C](./Figures/C_fig.png "Configuration C")

Using small boxes $[x - 0.1, x + 0.1]$, modify the code given in section A1 to show that the probability to sample configurations A, B, and C are the same (within the numerical precision), with

A = [(0.25, 0.25), (0.25, 0.75), (0.75, 0.25), (0.75,0.75)]

B = [(0.20, 0.20), (0.20, 0.80), (0.75, 0.25), (0.75,0.75)]

C = [(0.30, 0.20), (0.30, 0.80), (0.70, 0.20), (0.70,0.70)]

Run your program for a disk radius $\sigma=0$  and for $\sigma=0.1$. In two sentences,
explain what this program does and write down the obtained frequencies for configurations A, B, and C,
with four digits (e.g. 0.003) and provide a short explanation.

## B2
Provide an analytical expression for these probabilities, for both cases $\sigma=0$ and for $\sigma=0.1$, and check it
against the results of section (B1). Note that for $\sigma=0.1$, you need to know the acceptance ratio of the algorithm, and you need to modify your program to obtain it. Explain your analytical formula both for $\sigma=0$ and for $\sigma=0.1$ and compare it to your results from part (B1).